In [1]:
data_root = '/home/scott/Documents/SPEC_Spider'

In [2]:
!tree /home/scott/Documents/SPEC_Spider

/home/scott/Documents/SPEC_Spider
├── cpu
│   ├── cpu2006
│   │   ├── SPECfp.csv
│   │   ├── SPECfp_rate.csv
│   │   ├── SPECint.csv
│   │   └── SPECint_rate.csv
│   └── cpu2017
│       ├── CFP2017_rate.csv
│       ├── CFP2017_speed.csv
│       ├── CINT2017_rate.csv
│       └── CINT2017_speed.csv
├── java
│   ├── jbb2015
│   │   ├── SPECjbb2015-Composite.csv
│   │   ├── SPECjbb2015-Distributed.csv
│   │   └── SPECjbb2015-MultiJVM.csv
│   └── jvm2008
│       └── jvm2008.csv
├── jbb2015.csv
└── power
    └── ssj2008.csv

7 directories, 14 files


In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
c2017_rfp = pd.read_csv(f"{data_root}/cpu/cpu2017/CFP2017_rate.csv")
c2017_sfp = pd.read_csv(f"{data_root}/cpu/cpu2017/CFP2017_speed.csv")
c2017_rint = pd.read_csv(f"{data_root}/cpu/cpu2017/CINT2017_rate.csv")
c2017_sint = pd.read_csv(f"{data_root}/cpu/cpu2017/CINT2017_speed.csv")

c2006_rfp = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECfp_rate.csv")
c2006_sfp = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECfp.csv")
c2006_rint = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECint_rate.csv")
c2006_sint = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECint.csv")

j2015_com = pd.read_csv(f"{data_root}/java/jbb2015/SPECjbb2015-Composite.csv")
j2015_dis = pd.read_csv(f"{data_root}/java/jbb2015/SPECjbb2015-Distributed.csv")
j2015_mul = pd.read_csv(f"{data_root}/java/jbb2015/SPECjbb2015-MultiJVM.csv")

jvm2008 = pd.read_csv(f"{data_root}/java/jvm2008/jvm2008.csv")
ssj2008 = pd.read_csv(f"{data_root}/power/ssj2008.csv")

In [5]:
hw_vendors = set(c2017_rfp['Hardware Vendor'].unique().tolist())
hw_vendors |= set(c2017_sfp['Hardware Vendor'].unique().tolist())
hw_vendors |= set(c2017_rint['Hardware Vendor'].unique().tolist())
hw_vendors |= set(c2017_sint['Hardware Vendor'].unique().tolist())

hw_vendors = set(c2006_rfp['Hardware Vendor'].unique().tolist())
hw_vendors |= set(c2006_sfp['Hardware Vendor'].unique().tolist())
hw_vendors |= set(c2006_rint['Hardware Vendor'].unique().tolist())
hw_vendors |= set(c2006_sint['Hardware Vendor'].unique().tolist())
hw_vendors |= set(c2006_sint['Hardware Vendor'].unique().tolist())

hw_vendors |= set(j2015_com['Vendor'].unique().tolist())
hw_vendors |= set(j2015_mul['Vendor'].unique().tolist())
hw_vendors |= set(j2015_dis['Vendor'].unique().tolist())

hw_vendors |= set(jvm2008['HW vendor'].unique().tolist())

hw_vendors |= set(ssj2008['Hardware Vendor'].unique().tolist())

hw_vendors_df = pd.DataFrame(list(hw_vendors), columns=['Vendor'])

Hardware Vendor / Test Sponsor / Tested By / JVM Vendor

In [32]:
def clean_vendor(vendor):
    def _clear(pattern):
        return re.sub(re.compile(pattern, re.IGNORECASE), '', vendor)
    vendor = vendor.strip()
    patterns = [
        ' *[(].*[)]',
        ',* *Ltd\.*$|,* *Inc\.*$',
        ',* *Co\.*$|,* *Corporation\.*$|,* *Corparation\.*$|,* *Corp\.*$| Incoporated$| Incorporated$| Incorporation$',
        ' International$',
        ' Computer[s]*$',
        ' Technology$'
    ]
    for pattern in patterns:
        vendor = _clear(pattern)
    
    replace_pairs = [
        ('^Huawei', 'Huawei'),
        ('^ASUS', 'ASUS'),
        ('^acer', 'Acer'),
        ('^Hewlett[ -]*Packard', 'HPE'),
        ('^Inspur', 'Inspur'),
        ('H3C', 'H3C'),
        ('^Giga[ -]*byte', 'Gigabyte'),
        ('^Fujitsu', 'Fujitsu'),
        ('^Hitachi', 'Hitachi'),
        ('^Lenovo', 'Lenovo'),
        ('^Quanta', 'Quanta'),
        ('^Super[ -]*Micro', 'SuperMicro'),
        ('^UNIWIDE', 'Uniwide'),
        ('^Wizbrain', 'Wizbrain'),
        ('^ScaleMP', 'ScaleMP'),
        ('^AMD', 'AMD'),
        ('Advanced Micro Devices', 'AMD'),
        ('^Hewelett-Packard', 'HPE'),
        ('^Oracl', 'Oracle'),
        ('^BEA', 'BEA'),
        ('^OpenJDK', 'OpenJDK'),
    ]
    for pair in replace_pairs:
        if len(re.findall(re.compile(pair[0], re.IGNORECASE), vendor)):
            vendor = pair[1]
    
    return vendor

In [7]:
np.sort(hw_vendors_df['Vendor'].apply(lambda x: clean_vendor(x)).unique())

array(['3Score', 'ACTION S.A.', 'ASUS', 'ATEC', 'Acer', 'Apple',
       'BOXX Technologies', 'Bull SAS', 'Cisco Systems', 'ClearCube',
       'Clevo', 'Cryo Performance Computing', 'DIGI.', 'Dell', 'Digi',
       'Digi Intermational', 'E4 Computer Engineering S.p.A.', 'EXO S.A.',
       'FORMAT', 'Fujitsu', 'Gigabyte', 'H3C', 'HPE', 'Hitachi', 'Huawei',
       'Hypertechnologies Ciara', 'IBM', 'IWILL', 'Incom S.A.', 'Inspur',
       'Intel', 'Itautec', 'J.P Sa Couto S.A.', 'KTNF', 'LTechKorea',
       'Lenovo', 'M Computers s.r.o.', 'MSI', 'Maxdata AG', 'Microsoft',
       'NEC', 'NTT System S. A.', 'Nettars tech', 'Nettrix', 'Nokia',
       'Oracle', 'PC Factory S.A.', 'Quanta',
       'RDing Technology Limited Company', 'SGI', 'ScaleMP', 'Shuttle',
       'Sugon', 'Sun Microsystems', 'SuperMicro', 'TAEJIN T&S',
       'TEMPERATURE@LERT', 'TEMPERATURE@LERT.', 'TempAlert',
       'Temperature@lert', 'Tyan', 'Unisys', 'Uniwide', 'Wipro Limited',
       'Wizbrain', 'YOYOtech', 'Yadro', '

In [8]:
sponsors = set(c2017_rfp['Test Sponsor'].unique().tolist())
sponsors |= set(c2017_sfp['Test Sponsor'].unique().tolist())
sponsors |= set(c2017_rint['Test Sponsor'].unique().tolist())
sponsors |= set(c2017_sint['Test Sponsor'].unique().tolist())

sponsors = set(c2006_rfp['Test Sponsor'].unique().tolist())
sponsors |= set(c2006_sfp['Test Sponsor'].unique().tolist())
sponsors |= set(c2006_rint['Test Sponsor'].unique().tolist())
sponsors |= set(c2006_sint['Test Sponsor'].unique().tolist())
sponsors |= set(c2006_sint['Test Sponsor'].unique().tolist())

sponsors |= set(j2015_com['Test Sponsor'].unique().tolist())
sponsors |= set(j2015_mul['Test Sponsor'].unique().tolist())
sponsors |= set(j2015_dis['Test Sponsor'].unique().tolist())

# hw_vendors |= set(jvm2008['HW vendor'].unique().tolist())

sponsors |= set(ssj2008['Test Sponsor'].unique().tolist())

sponsor_df = pd.DataFrame(list(sponsors), columns=['Sponsor'])

In [9]:
sponsor_df['Sponsor'].unique()

array(['EXO S.A.',
       'Inspur(Beijing) Electronic Information Industry Co., Ltd',
       'Hewlett-Packard Company', 'DEPO Computers',
       'Verari Systems, Inc.', 'SGI', 'HITACHI', 'M Computers s.r.o.',
       'QLogic [formerly PathScale]', 'Fujitsu Limited', 'HPE',
       'Future Publishing Ltd.', 'BOXX Technologies, Inc.',
       'Nokia Corporation', 'Hewlett Packard Enterprise',
       'Sun Microsystems, Inc.', 'Quanta Computer Inc.',
       'E4 Computer Engineering S.p.A.', 'Sun Microsystems', 'H3C',
       'ScaleMP', 'Acer Incorporated', 'Inspur PowerSystem Corporation',
       'ACTION S.A.', 'Hypertechnologies Ciara, Inc', 'Cisco Systems',
       'Unisys Corporation', 'Incom S.A.',
       'Telecommunications Technology Association', 'Hitachi, Ltd.',
       'Sugon', 'IBM Corporation', 'Quanta Cloud Technology', 'Bull SAS',
       'ASUSTeK Company Inc.', 'ZT Systems', 'Maxdata',
       'NTT System S. A.', 'GIGA-BYTE TECHNOLOGY CO., LTD.',
       'NEC Corporation', 'Apple, Inc

In [10]:
np.sort(sponsor_df['Sponsor'].apply(lambda x: clean_vendor(x)).unique())

array(['ACTION S.A.', 'AMD', 'ASUS', 'Acer', 'Apple', 'BOXX Technologies',
       'Bull SAS', 'Cisco Systems', 'ClearCube',
       'Cryo Performance Computing', 'DEPO', 'Dell',
       'E4 Computer Engineering S.p.A.', 'EXO S.A.', 'FORMAT',
       'Five9 Network Systems', 'Fujitsu', 'Future Publishing',
       'Gigabyte', 'H3C', 'HPE', 'Hitachi', 'Huawei',
       'Hypertechnologies Ciara', 'IBM', 'Incom S.A.', 'Inspur', 'Intel',
       'Itautec', 'J.P Sa Couto S.A.',
       'Japan Advanced Institute of Science and', 'Lenovo',
       'M Computers s.r.o.', 'Maxdata', 'NEC', 'NTT System S. A.',
       'Nettrix', 'Nokia', 'Oracle', 'PC Factory S.A.', "Plat'Home",
       'Powerleader Company', 'QLogic [formerly PathScale]', 'Quanta',
       'SGI', 'ScaleMP', 'Sugon', 'Sun Microsystems', 'SuperMicro',
       'Telecommunications Technology Association', 'The Portland Group',
       'Unisys', 'Verari Systems', 'Wipro Limited', 'Yadro', 'ZT Systems',
       'ZTE'], dtype=object)

In [11]:
submitter = set(c2017_rfp['Tested By'].unique().tolist())
submitter |= set(c2017_sfp['Tested By'].unique().tolist())
submitter |= set(c2017_rint['Tested By'].unique().tolist())
submitter |= set(c2017_sint['Tested By'].unique().tolist())

submitter = set(c2006_rfp['Tested By'].unique().tolist())
submitter |= set(c2006_sfp['Tested By'].unique().tolist())
submitter |= set(c2006_rint['Tested By'].unique().tolist())
submitter |= set(c2006_sint['Tested By'].unique().tolist())
submitter |= set(c2006_sint['Tested By'].unique().tolist())

submitter |= set(j2015_com['Tested by'].unique().tolist())
submitter |= set(j2015_mul['Tested by'].unique().tolist())
submitter |= set(j2015_dis['Tested by'].unique().tolist())

submitter |= set(jvm2008['Submitter'].unique().tolist())

submitter |= set(ssj2008['Tested By'].unique().tolist())

submitter_df = pd.DataFrame(list(submitter), columns=['Submitter'])

In [12]:
submitter_df['Submitter'].unique()

array(['EXO S.A.',
       'Inspur(Beijing) Electronic Information Industry Co., Ltd',
       'Hewlett-Packard Company', 'DEPO Computers',
       'Verari Systems, Inc.', 'SGI', 'HITACHI', 'M Computers s.r.o.',
       'Fujitsu Limited', 'HPE', 'Future Publishing Ltd.',
       'BOXX Technologies, Inc.', 'Krzysztof Gierczyk',
       'Nokia Corporation', 'Hewlett Packard Enterprise',
       'Sun Microsystems, Inc.', 'Quanta Computer Inc.',
       'E4 Computer Engineering S.p.A.', 'Sun Microsystems',
       'Huawei Technologies Co', ' Huawei', 'H3C', 'ScaleMP',
       'Acer Incorporated', 'Inspur PowerSystem Corporation',
       ' Hewlett-Packard Company', 'ACTION S.A.',
       'Hypertechnologies Ciara, Inc', 'Cisco Systems',
       'Unisys Corporation', 'Incom S.A.', 'Piotr Nowicki',
       'Telecommunications Technology Association', 'Hitachi, Ltd.',
       'Sugon', 'IBM Corporation', 'Francesca Tartaglione',
       'Quanta Cloud Technology', 'Sun Microsystems Inc.', 'Bull SAS',
       'AM

In [13]:
np.sort(submitter_df['Submitter'].apply(lambda x: clean_vendor(x)).unique())

array(['ACTION S.A.', 'AMD', 'ASUS', 'Acer', 'Apple', 'Apple, ',
       'BOXX Technologies', 'Bull SAS', 'Cisco Systems', 'ClearCube',
       'Cryo Performance Computing', 'DEPO', 'Dell',
       'E4 Computer Engineering S.p.A.', 'EXO S.A.', 'FORMAT',
       'Five9 Network Systems', 'Francesca Tartaglione', 'Fujitsu',
       'Future Publishing', 'Gigabyte', 'H3C', 'HPE', 'Hitachi', 'Huawei',
       'Hypertechnologies Ciara', 'IBM', 'Incom S.A.', 'Inspur', 'Intel',
       'Itautec', 'J.P Sa Couto S.A.', 'Krzysztof Gierczyk', 'Lenovo',
       'M Computers s.r.o.', 'Maxdata', 'NEC', 'NTT System S. A.',
       'Nettrix', 'Nokia', 'Oracle', 'PC Factory S.A.', 'Piotr Nowicki',
       "Plat'Home", 'Powerleader Company', 'Principled Technologies',
       'QLogic', 'Quanta', 'SGI', 'ScaleMP', 'Sugon', 'Sun Microsystems',
       'SuperMicro', 'Telecommunications Technology Association',
       'The Portland Group', 'Unisys', 'Verari Systems', 'Wipro Limited',
       'Yadro', 'ZTE'], dtype=object)

---

In [22]:
jvm_vendors = set(j2015_com['JVM Vendor'].unique().tolist())
jvm_vendors |= set(j2015_mul['JVM Vendor'].unique().tolist())
jvm_vendors |= set(j2015_dis['JVM Vendor'].unique().tolist())

jvm_vendors |= set(jvm2008['JVM Vendor'].unique().tolist())

jvm_vendors |= set(ssj2008['JVM Vendor'].unique().tolist())

jvm_vendor_df = pd.DataFrame(list(jvm_vendors), columns=['Vendor'])

In [23]:
jvm_vendor_df['Vendor'].unique()

array(['OpenJDK community', 'IBM Corparation', 'IBM Corp.', 'Oracle Inc.',
       'Red Hat, Inc.', 'Oracl', 'IBM Corporation',
       'Sun Microsystems Inc.', 'Oracle Corporation.', 'SUSE',
       'Apple Inc.', 'Bea Systems, Inc', 'Oracle', 'BEA Systems,Inc.',
       'Sun Microsystems, Inc.', 'OpenJDK (openjdk.java.net)',
       'Oracle Corp.', 'OpenJDK', 'BEA System, Inc.', 'BEA Systems, Inc.',
       'Oracle corporation', 'BEA Systems, Inc', 'Oracle Corporation'],
      dtype=object)

In [33]:
np.sort(jvm_vendor_df['Vendor'].apply(lambda x: clean_vendor(x)).unique())

array(['Apple', 'BEA', 'IBM', 'OpenJDK', 'Oracle', 'Red Hat', 'SUSE',
       'Sun Microsystems'], dtype=object)